# AutoInland Vehicle Insurance Claim Challenge Modelling Notebook - Python

This notebook covers Data Modelling for AutoInland Insurance Claim Prediction:
- Loading the prepared data
- Feature selection
- Model Evaluation- Using Stratified Kfold.
- Final Model Building and generating predictions

In [1]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings

from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, auc, mean_squared_error, roc_curve, confusion_matrix, precision_score, recall_score, f1_score,\
log_loss, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
import warnings
from scipy.stats import chi2_contingency


from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import  DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split , StratifiedKFold  , StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import os
import random
from sklearn.metrics import f1_score, fbeta_score, make_scorer, balanced_accuracy_score, accuracy_score, roc_auc_score
pd.set_option("display.max_columns",30)

# random.seed(40)
# np.random.rand(40)

rand = 40



In [2]:
data = pd.read_csv('all_data_processed_finaleng.csv')
ss = pd.read_csv('SampleSubmission_Insurance.csv')

data = data.drop(['Unnamed: 0'], axis = 1)

In [3]:
data.head()

,ID,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,target,tag,Policy Start Date year,Policy Start Date month,Policy Start Date day,Policy Start Date dayofweek,...,Red,Ash,Wine,Brown,Black,Car_Category_new,age_group,age_group_risk,Policy_brackets,State_zone,is_rainfall,is_peak,risk_index,Gender_age_risk,No Pol/Age
0,ID_0040R73,Male,30.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,5,14,4,...,0,0,0,0,1,Car,25-34,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.033333
1,ID_0046BNK,Female,79.0,1,JEEP,TOYOTA,NaN,NaN,Car Classic,1.0,train,2010.0,11,29,0,...,0,0,0,0,0,JEEP,65+,High-risk elderly,12months,NaN,0,0,0,Female- High-risk elderly,0.012658
2,ID_005QMC3,Male,43.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,3,21,6,...,1,0,0,0,0,Car,35-44,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.023256
3,ID_0079OHW,Male,NaN,1,NaN,NaN,NaN,NaN,CarSafe,0.0,train,2010.0,8,21,5,...,0,0,0,0,0,NaN,NaN,NaN,12months,NaN,0,0,0,NaN,NaN
4,ID_00BRP63,NaN,20.0,3,NaN,NaN,ikeja,Lagos,Muuve,1.0,train,2010.0,8,29,6,...,0,0,0,0,0,NaN,18-20,high-risk youngsters,6months,South_West,1,0,1,NaN,0.150000


In [4]:
data.columns

Index(['ID', 'Gender', 'Age', 'No_Pol', 'Car_Category', 'Subject_Car_Make',
       'LGA_Name', 'State', 'ProductName', 'target', 'tag',
       'Policy Start Date year', 'Policy Start Date month',
       'Policy Start Date day', 'Policy Start Date dayofweek',
       'Policy End Date year', 'Policy End Date month', 'Policy End Date day',
       'Policy End Date dayofweek', 'weekinmonth', 'Policy_delta',
       'Next 1 Months', 'Next 1 Quarters', 'Next 2 Months', 'Next 2 Quarters',
       'Next 3 Months', 'Next 3 Quarters', 'summer', 'winter', 'spring',
       'autumn', 'Policy_isfestive', 'color_group', 'Silver', 'Blue', 'Gold',
       'Light', 'Green', 'Orange', 'Yellow', 'Grey', 'White', 'Bright', 'Dark',
       'Purple', 'Cream', 'Red', 'Ash', 'Wine', 'Brown', 'Black',
       'Car_Category_new', 'age_group', 'age_group_risk', 'Policy_brackets',
       'State_zone', 'is_rainfall', 'is_peak', 'risk_index', 'Gender_age_risk',
       'No Pol/Age'],
      dtype='object')

In [5]:
data.dtypes

ID                  object
Gender              object
Age                float64
No_Pol               int64
Car_Category        object
                    ...   
is_rainfall          int64
is_peak              int64
risk_index           int64
Gender_age_risk     object
No Pol/Age         float64
Length: 61, dtype: object

In [6]:
data.columns

Index(['ID', 'Gender', 'Age', 'No_Pol', 'Car_Category', 'Subject_Car_Make',
       'LGA_Name', 'State', 'ProductName', 'target', 'tag',
       'Policy Start Date year', 'Policy Start Date month',
       'Policy Start Date day', 'Policy Start Date dayofweek',
       'Policy End Date year', 'Policy End Date month', 'Policy End Date day',
       'Policy End Date dayofweek', 'weekinmonth', 'Policy_delta',
       'Next 1 Months', 'Next 1 Quarters', 'Next 2 Months', 'Next 2 Quarters',
       'Next 3 Months', 'Next 3 Quarters', 'summer', 'winter', 'spring',
       'autumn', 'Policy_isfestive', 'color_group', 'Silver', 'Blue', 'Gold',
       'Light', 'Green', 'Orange', 'Yellow', 'Grey', 'White', 'Bright', 'Dark',
       'Purple', 'Cream', 'Red', 'Ash', 'Wine', 'Brown', 'Black',
       'Car_Category_new', 'age_group', 'age_group_risk', 'Policy_brackets',
       'State_zone', 'is_rainfall', 'is_peak', 'risk_index', 'Gender_age_risk',
       'No Pol/Age'],
      dtype='object')

In [7]:
data['State_Statezone'] = data['State'] + "_" + data['State_zone']

In [8]:
drop_cols = ['Next 2 Quarters', 'Next 3 Quarters', 'Next 1 Quarters', \
            'Policy Start Date year', 'Car_Category_new',\
        'Policy End Date month', 'Policy Start Date day', 'Policy Start Date month',\
            'Next 1 Months', 'Next 2 Months', 'Next 3 Months', 'Next 2 Quarters', 
           'age_group_risk', 'Car_Category']
             
           

In [9]:
data = data.drop(drop_cols, axis = 1)

In [10]:
data = data.drop(['Yellow', 'Orange', 'Light', 'Dark', 'Bright', 'Purple', 'Cream'], axis = 1)

In [11]:
#Extract categorical

cat_cols = []

for col in data.columns:
    
    if data[col].dtype == 'O':
        
        cat_cols.append(col)

In [12]:
cat_cols.remove('tag')
cat_cols.remove('ID')

In [13]:
data.columns

Index(['ID', 'Gender', 'Age', 'No_Pol', 'Subject_Car_Make', 'LGA_Name',
       'State', 'ProductName', 'target', 'tag', 'Policy Start Date dayofweek',
       'Policy End Date year', 'Policy End Date day',
       'Policy End Date dayofweek', 'weekinmonth', 'Policy_delta', 'summer',
       'winter', 'spring', 'autumn', 'Policy_isfestive', 'color_group',
       'Silver', 'Blue', 'Gold', 'Green', 'Grey', 'White', 'Red', 'Ash',
       'Wine', 'Brown', 'Black', 'age_group', 'Policy_brackets', 'State_zone',
       'is_rainfall', 'is_peak', 'risk_index', 'Gender_age_risk', 'No Pol/Age',
       'State_Statezone'],
      dtype='object')

In [14]:
data.isnull().sum()

ID                                0
Gender                         1246
Age                             589
No_Pol                            0
Subject_Car_Make               2785
LGA_Name                       7145
State                          7145
ProductName                       0
target                         1202
tag                               0
Policy Start Date dayofweek       0
Policy End Date year              6
Policy End Date day               0
Policy End Date dayofweek         0
weekinmonth                       0
Policy_delta                     11
summer                            0
winter                            0
spring                            0
autumn                            0
Policy_isfestive                  0
color_group                    8214
Silver                            0
Blue                              0
Gold                              0
Green                             0
Grey                              0
White                       

In [15]:
for col in cat_cols:

    data[col] = pd.factorize(data[col])[0]

In [16]:
for col in cat_cols:
    
    print(data[col].value_counts())

 0    8356
 1    3679
-1    1246
Name: Gender, dtype: int64
 0     5466
-1     2785
 6     1146
 2      654
 7      577
 3      512
 14     375
 9      307
 11     275
 10     133
 4      129
 13     121
 15     102
 8       69
 17      68
 24      66
 23      47
 26      47
 20      43
 1       42
 21      35
 29      33
 18      30
 12      29
 25      26
 5       21
 33      18
 28      17
 22      16
 16      12
 30       9
 40       8
 32       8
 35       7
 19       6
 36       6
 37       6
 38       6
 41       5
 27       5
 31       5
 39       5
 34       4
Name: Subject_Car_Make, dtype: int64
-1      7145
 3      1284
 0       625
 11      306
 8       257
        ... 
 74        4
 49        4
 58        4
 65        4
 127       4
Name: LGA_Name, Length: 131, dtype: int64
-1     7145
 0     4073
 1      693
 4      340
 6      204
 2      138
 3      120
 5      117
 10      78
 8       56
 13      50
 7       49
 12      47
 17      43
 14      24
 11      24
 15      1

In [17]:
data.head()

,ID,Gender,Age,No_Pol,Subject_Car_Make,LGA_Name,State,ProductName,target,tag,Policy Start Date dayofweek,Policy End Date year,Policy End Date day,Policy End Date dayofweek,weekinmonth,...,White,Red,Ash,Wine,Brown,Black,age_group,Policy_brackets,State_zone,is_rainfall,is_peak,risk_index,Gender_age_risk,No Pol/Age,State_Statezone
0,ID_0040R73,0,30.0,1,0,-1,-1,0,0.0,train,4,2011.0,13,4,3,...,0,0,0,0,0,1,0,0,-1,0,1,2,0,0.033333,-1
1,ID_0046BNK,1,79.0,1,0,-1,-1,0,1.0,train,0,2011.0,28,0,4,...,0,0,0,0,0,0,1,0,-1,0,0,0,1,0.012658,-1
2,ID_005QMC3,0,43.0,1,0,-1,-1,0,0.0,train,6,2011.0,20,6,4,...,0,1,0,0,0,0,2,0,-1,0,1,2,0,0.023256,-1
3,ID_0079OHW,0,NaN,1,-1,-1,-1,1,0.0,train,5,2011.0,20,5,4,...,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,NaN,-1
4,ID_00BRP63,-1,20.0,3,-1,0,0,2,1.0,train,6,2010.0,31,4,4,...,0,0,0,0,0,0,3,1,0,1,0,1,-1,0.150000,0


In [18]:
data.columns

Index(['ID', 'Gender', 'Age', 'No_Pol', 'Subject_Car_Make', 'LGA_Name',
       'State', 'ProductName', 'target', 'tag', 'Policy Start Date dayofweek',
       'Policy End Date year', 'Policy End Date day',
       'Policy End Date dayofweek', 'weekinmonth', 'Policy_delta', 'summer',
       'winter', 'spring', 'autumn', 'Policy_isfestive', 'color_group',
       'Silver', 'Blue', 'Gold', 'Green', 'Grey', 'White', 'Red', 'Ash',
       'Wine', 'Brown', 'Black', 'age_group', 'Policy_brackets', 'State_zone',
       'is_rainfall', 'is_peak', 'risk_index', 'Gender_age_risk', 'No Pol/Age',
       'State_Statezone'],
      dtype='object')

In [19]:
train = data[data['tag'] == 'train']

test = data[data['tag'] == 'test']


train = train.drop(['tag', 'ID'], axis = 1)

test = test.drop(['tag'], axis = 1)

In [20]:
train.shape, test.shape

((12079, 40), (1202, 41))

In [21]:
y = train['target'].astype(int)

X = train.drop(['target'], axis = 1)

### Modelling

In [22]:
X.columns

Index(['Gender', 'Age', 'No_Pol', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'Policy Start Date dayofweek', 'Policy End Date year',
       'Policy End Date day', 'Policy End Date dayofweek', 'weekinmonth',
       'Policy_delta', 'summer', 'winter', 'spring', 'autumn',
       'Policy_isfestive', 'color_group', 'Silver', 'Blue', 'Gold', 'Green',
       'Grey', 'White', 'Red', 'Ash', 'Wine', 'Brown', 'Black', 'age_group',
       'Policy_brackets', 'State_zone', 'is_rainfall', 'is_peak', 'risk_index',
       'Gender_age_risk', 'No Pol/Age', 'State_Statezone'],
      dtype='object')

In [23]:
cat_cols = cat_cols + ['risk_index']

In [24]:
#Cross-Validation LGBM routine

def lgb_crossval(model):

    rand = 40

    fold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= rand)

    i = 1

    f1_bin = []
    
    f1_mac = []

    acc = []

    bal_acc = []
    
    mcc = []
    
    test_new = test[X.columns]
    
    predictions = []
    
    
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(X.shape[1])

    




    for train_index, test_index in fold.split(X, y):
        
    
        print('fold n°: ', i)

        x_data, x_val = X.iloc[train_index], X.iloc[test_index]

        y_data, y_val = y.iloc[train_index], y.iloc[test_index]
    
    
    
   
        
        model.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
    
        y_train_pred = model.predict(x_data)
        
        y_test_pred = model.predict(x_val)
        
        predictions.append(model.predict(test_new))   
        
        
        feature_importances += model.feature_importances_ 
    
        print('f1_binary train', f1_score(y_data, y_train_pred, average = 'binary'))
        print('f1_binary test',  f1_score(y_val, y_test_pred, average = 'binary'))
        
        print('f1_macro train', f1_score(y_data, y_train_pred, average = 'macro'))
        print('f1_macro test',  f1_score(y_val, y_test_pred, average = 'macro'))
    
        print('accuracy train', accuracy_score(y_data, y_train_pred))
        print('accuracy test',  accuracy_score(y_val, y_test_pred))
    
        print('balanced accuracy train', balanced_accuracy_score(y_data, y_train_pred))
        print('balanced accuracy test',  balanced_accuracy_score(y_val, y_test_pred))
    
    
        print('mcc train', matthews_corrcoef(y_data, y_train_pred))
        print('mcc test', matthews_corrcoef(y_val, y_test_pred))
    
    
    
        f1_bin.append(f1_score(y_val, y_test_pred, average = 'binary'))
        
        f1_mac.append(f1_score(y_val, y_test_pred, average = 'macro'))
        
        acc.append(accuracy_score(y_val, y_test_pred))
    
        bal_acc.append(balanced_accuracy_score(y_val, y_test_pred))
        
        mcc.append(matthews_corrcoef(y_val, y_test_pred))


        i = i + 1
        
    print('Mean f1 binary test: ', np.mean(f1_bin)) 
    
    print('Mean f1 macro test: ', np.mean(f1_mac))

    print('Mean Acc test: ', np.mean(acc))

    print('Mean Bal Acc test: ', np.mean(bal_acc))
    
    print('Mean Mcc test: ', np.mean(mcc))
    
    
    print('STD f1 binary test: ', np.std(f1_bin)) 
    
    print('STD Mean acc test: ', np.std(acc))
    
    return predictions, feature_importances/10, f1_bin

**LGBM 1**

In [25]:
lgb_model = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, \
                               boosting = 'dart', num_leaves = 60, \
                               importance_type = 'gain', reg_lambda = 0.5,
                          random_state = 40)


In [26]:
predictions, importances, f1_scores = lgb_crossval(lgb_model)

fold n°:  1


C:\Users\Damola\Anaconda3\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


f1_binary train 0.7811568276684556
f1_binary test 0.3863636363636364
f1_macro train 0.8706197451372515
f1_macro test 0.6408562367864694
accuracy train 0.9324809125195475
accuracy test 0.8211920529801324
balanced accuracy train 0.9616148938395566
balanced accuracy test 0.669101761442891
mcc train 0.7692191830647067
mcc test 0.29169336395966006
fold n°:  2
f1_binary train 0.7896323086196503
f1_binary test 0.4096385542168675
f1_macro train 0.8758734708534639
f1_macro test 0.6577943250930787
accuracy train 0.9357924753932481
accuracy test 0.8377483443708609
balanced accuracy train 0.9634975420981069
balanced accuracy test 0.6785090991663152
mcc train 0.7776649209213398
mcc test 0.32078002218658214
fold n°:  3
f1_binary train 0.790108564535585
f1_binary test 0.38526912181303113
f1_macro train 0.8761679260320387
f1_macro test 0.6400412501939611
accuracy train 0.9359764511084537
accuracy test 0.820364238410596
balanced accuracy train 0.9636021336680264
balanced accuracy test 0.668631394556719

**LGBM 2**

In [27]:
#LGB JULY DART ALPHA

lgb_model_2 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_alpha = 0.5,
                            random_state = 40, feature_fraction = 0.9)

In [28]:
predictions, importances, f1_scores = lgb_crossval(lgb_model_2)

fold n°:  1


C:\Users\Damola\Anaconda3\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


f1_binary train 0.7856071964017991
f1_binary test 0.4034582132564842
f1_macro train 0.8733816391635768
f1_macro test 0.6517049403643466
accuracy train 0.9342286818140005
accuracy test 0.8286423841059603
balanced accuracy train 0.9626085137537914
balanced accuracy test 0.6792908813702274
mcc train 0.7736507350027773
mcc test 0.3122943708771815
fold n°:  2
f1_binary train 0.7908240265620283
f1_binary test 0.40707964601769914
f1_macro train 0.8766101249528357
f1_macro test 0.655152726234656
accuracy train 0.9362524146812621
accuracy test 0.8336092715231788
balanced accuracy train 0.9637590210229056
balanced accuracy test 0.6791351737113569
mcc train 0.778854540942073
mcc test 0.3171331605759981
fold n°:  3
f1_binary train 0.7941800545619885
f1_binary test 0.39215686274509803
f1_macro train 0.8786819591792754
f1_macro test 0.6433829481282557
accuracy train 0.9375402446877013
accuracy test 0.820364238410596
balanced accuracy train 0.9644911620123418
balanced accuracy test 0.6745872125085153

**ENSEMBLE**

In [29]:
#Create LIGHTGBM ENSEMBLE

def ensemb_crossval():

    rand = 40

    fold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= rand)

    i = 1

    f1_bin = []
    
    f1_mac = []

    acc = []

    bal_acc = []
    
    mcc = []
    
    test_new = test[X.columns]
    
    predictions = []
    
    
    

    




    for train_index, test_index in fold.split(X, y):
        
    
        print('fold n°: ', i)
        
        predictions_train = list()

        predictions_test = list()

        x_data, x_val = X.iloc[train_index], X.iloc[test_index]

        y_data, y_val = y.iloc[train_index], y.iloc[test_index]
        
        
        
        lgb_model_1 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_lambda = 0.5,
                          seed = 40)
        
        
        lgb_model_1.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
        
    
        y_train_pred_lgb1 = lgb_model_1.predict_proba(x_data)
        
        y_val_pred_lgb1 = lgb_model_1.predict_proba(x_val)
        
        y_test_pred_lgb1 = lgb_model_1.predict_proba(test_new)
        
        
        
        
        lgb_model_2 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_alpha = 0.5, feature_fraction = 0.9,
                            seed = 40)
        
        lgb_model_2.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
        
    
        y_train_pred_lgb2 = lgb_model_2.predict_proba(x_data)
        
        y_val_pred_lgb2 = lgb_model_2.predict_proba(x_val)
        
        y_test_pred_lgb2 = lgb_model_2.predict_proba(test_new)
    
    
    
        y_train_pred = ((1.5*(y_train_pred_lgb1)) + (1.0*(y_train_pred_lgb2)))/2
        
        y_val_pred = ((1.5*(y_val_pred_lgb1)) + (1.0*(y_val_pred_lgb2)))/2
        
        y_test_pred = ((1.5*(y_test_pred_lgb1)) + (1.0*(y_test_pred_lgb2)))/2
        
        
        y_train_pred = np.argmax(y_train_pred, axis = 1).tolist()
        
        y_val_pred = np.argmax(y_val_pred, axis = 1).tolist()
        
        y_test_pred = np.argmax(y_test_pred, axis = 1).tolist()
        
        
        
        
        
        predictions.append(y_test_pred)
        
        
        
    
        
        print('f1_binary train', f1_score(y_data, y_train_pred, average = 'binary'))
        print('f1_binary test',  f1_score(y_val, y_val_pred, average = 'binary'))
        
        print('f1_macro train', f1_score(y_data, y_train_pred, average = 'macro'))
        print('f1_macro test',  f1_score(y_val, y_val_pred, average = 'macro'))
    
        print('accuracy train', accuracy_score(y_data, y_train_pred))
        print('accuracy test',  accuracy_score(y_val, y_val_pred))
    
        print('balanced accuracy train', balanced_accuracy_score(y_data, y_train_pred))
        print('balanced accuracy test',  balanced_accuracy_score(y_val, y_val_pred))
    
        print('mcc train', matthews_corrcoef(y_data, y_train_pred))
        print('mcc test', matthews_corrcoef(y_val, y_val_pred))
    
    
    
        f1_bin.append(f1_score(y_val, y_val_pred, average = 'binary'))
        
        f1_mac.append(f1_score(y_val, y_val_pred, average = 'macro'))
        
        acc.append(accuracy_score(y_val, y_val_pred))
    
        bal_acc.append(balanced_accuracy_score(y_val, y_val_pred))
        
        mcc.append(matthews_corrcoef(y_val, y_val_pred))


        i = i + 1
        
    print('Mean f1 binary test: ', np.mean(f1_bin)) 
    
    print('Mean f1 macro test: ', np.mean(f1_mac))

    print('Mean Acc test: ', np.mean(acc))

    print('Mean Bal Acc test: ', np.mean(bal_acc))
    
    print('Mean Mcc test: ', np.mean(mcc))
    
    
    return predictions


In [30]:
predictions = ensemb_crossval()

fold n°:  1


C:\Users\Damola\Anaconda3\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


f1_binary train 0.7834928229665072
f1_binary test 0.3953488372093023
f1_macro train 0.8720703597384987
f1_macro test 0.647481368411601
accuracy train 0.9334007910955754
accuracy test 0.8278145695364238
balanced accuracy train 0.9621378516891539
balanced accuracy test 0.6728646965322607
mcc train 0.7715444328814227
mcc test 0.3029108403478644
fold n°:  2
f1_binary train 0.7915407854984895
f1_binary test 0.41194029850746267
f1_macro train 0.8770529448325781
f1_macro test 0.6586371362792
accuracy train 0.9365283782540704
accuracy test 0.8369205298013245
balanced accuracy train 0.9639159083777847
balanced accuracy test 0.6810166412560418
mcc train 0.7795702918755294
mcc test 0.32318295078980913
fold n°:  3
f1_binary train 0.7944208611279564
f1_binary test 0.3909348441926346
f1_macro train 0.8788304696010634
f1_macro test 0.6433588423580721
accuracy train 0.937632232545304
accuracy test 0.8220198675496688
balanced accuracy train 0.9645434577973016
balanced accuracy test 0.67255003730496
mcc

In [31]:
feature_importances = pd.DataFrame({'feature': list(X.columns), 'importance': importances}).sort_values('importance', 
                                    ascending = True)
    

In [32]:
feature_importances

,feature,importance
31,Policy_brackets,56.275775
22,Green,84.708030
25,Red,94.598651
26,Ash,142.920188
28,Brown,150.292137
38,State_Statezone,244.193954
24,White,245.980026
21,Gold,253.535471
27,Wine,397.419236
32,State_zone,525.154819


In [33]:
test_predictions = np.array(predictions)

import scipy
test_predictions = list(scipy.stats.mode(test_predictions)[0][0])

In [34]:
ss['target'] = test_predictions

In [36]:
ss.to_csv('single.csv',index=False)                                  
                                                                                

In [35]:
ss['target'].value_counts()

0    1066
1     136
Name: target, dtype: int64